# Recommendations with IBM

In this notebook, you will be putting your recommendation skills to use on real data from the IBM Watson Studio platform. 


You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project [RUBRIC](https://review.udacity.com/#!/rubrics/2322/view).  **Please save regularly.**

By following the table of contents, you will build out a number of different methods for making recommendations that can be used for different situations. 


## Table of Contents

I. [Exploratory Data Analysis](#Exploratory-Data-Analysis)<br>
II. [Rank Based Recommendations](#Rank)<br>
III. [User-User Based Collaborative Filtering](#User-User)<br>
IV. [Content Based Recommendations (EXTRA - NOT REQUIRED)](#Content-Recs)<br>
V. [Matrix Factorization](#Matrix-Fact)<br>
VI. [Extras & Concluding](#conclusions)

At the end of the notebook, you will find directions for how to submit your work.  Let's get started by importing the necessary libraries and reading in the data.

In [1]:
import nltk
import re

nltk.download('punkt')

from nltk.corpus import stopwords

nltk.download('stopwords')

from nltk.tokenize import word_tokenize

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np

import time

import project_tests as t
import pickle

import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /Users/saha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/saha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/saha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/saha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('data/user-item-interactions.csv')
df_content = pd.read_csv('data/articles_community.csv')
del df['Unnamed: 0']
del df_content['Unnamed: 0']

In [3]:
# Show df to get an idea of the data
df.head()

,article_id,title,email
0,1430.0,"using pixiedust for fast, flexible, and easier...",ef5f11f77ba020cd36e1105a00ab868bbdbf7fe7
1,1314.0,healthcare python streaming application demo,083cbdfa93c8444beaa4c5f5e0f5f9198e4f9e0b
2,1429.0,use deep learning for image classification,b96a4f2e92d8572034b1e9b28f9ac673765cd074
3,1338.0,ml optimization using cognitive assistant,06485706b34a5c9bf2a0ecdac41daf7e7654ceb7
4,1276.0,deploy your python model as a restful api,f01220c46fc92c6e6b161b1849de11faacd7ccb2


In [4]:
# Show df_content to get an idea of the data
df_content.head()

,doc_body,doc_description,doc_full_name,doc_status,article_id
0,Skip navigation Sign in SearchLoading...\r\n\r...,Detect bad readings in real time using Python ...,Detect Malfunctioning IoT Sensors with Streami...,Live,0
1,No Free Hunch Navigation * kaggle.com\r\n\r\n ...,"See the forest, see the trees. Here lies the c...",Communicating data science: A guide to present...,Live,1
2,☰ * Login\r\n * Sign Up\r\n\r\n * Learning Pat...,Here’s this week’s news in Data Science and Bi...,"This Week in Data Science (April 18, 2017)",Live,2
3,"DATALAYER: HIGH THROUGHPUT, LOW LATENCY AT SCA...",Learn how distributed DBs solve the problem of...,DataLayer Conference: Boost the performance of...,Live,3
4,Skip navigation Sign in SearchLoading...\r\n\r...,This video demonstrates the power of IBM DataS...,Analyze NY Restaurant data using Spark in DSX,Live,4


### <a class="anchor" id="Exploratory-Data-Analysis">Part I : Exploratory Data Analysis</a>

Use the dictionary and cells below to provide some insight into the descriptive statistics of the data.

`1.` What is the distribution of how many articles a user interacts with in the dataset?  Provide a visual and descriptive statistics to assist with giving a look at the number of times each user interacts with an article.  

In [5]:
user_interact = df.groupby(['email'])['article_id'].count()

user_interact_df = pd.DataFrame(user_interact).reset_index()

In [6]:
user_interact_df.head()

,email,article_id
0,0000b6387a0366322d7fbfc6434af145adf7fed1,13
1,001055fc0bb67f71e8fa17002342b256a30254cd,4
2,00148e4911c7e04eeff8def7bbbdaf1c59c2c621,3
3,001a852ecbd6cc12ab77a785efa137b2646505fe,6
4,001fc95b90da5c3cb12c501d201a915e4f093290,2


In [7]:
user_interact_df.rename(columns={'article_id':'views_by_user'},inplace=True)

In [8]:
trace = go.Histogram(
    x = user_interact_df.views_by_user,
    xbins=dict(
        start=np.min(user_interact_df.views_by_user),
        end=np.max(user_interact_df.views_by_user)
    ))

layout = go.Layout(title='Frequency of user-article interactions')
fig = go.Figure(data=go.Data([trace]),layout=layout)
iplot(fig)

In [9]:
user_interact_df.views_by_user.median(), user_interact_df.views_by_user.max()

(3.0, 364)

In [10]:
# Fill in the median and maximum number of user_article interactios below

# 50% of individuals interact with ____ number of articles or fewer. - 3.0
median_val = user_interact_df.views_by_user.median() 

# The maximum number of user-article interactions by any 1 user is ______. - 364
max_views_by_user = user_interact_df.views_by_user.max()

`2.` Explore and remove duplicate articles from the **df_content** dataframe.  

In [11]:
# Find and explore duplicate articles
(df_content.article_id.value_counts()>1).sum()

5

In [12]:
# Remove any rows that have the same article_id - only keep the first
df_content.drop_duplicates('article_id',keep='first',inplace=True)

In [13]:
(df_content.article_id.value_counts()>1).sum()

0

`3.` Use the cells below to find:

**a.** The number of unique articles that have an interaction with a user.  
**b.** The number of unique articles in the dataset (whether they have any interactions or not).<br>
**c.** The number of unique users in the dataset. (excluding null values) <br>
**d.** The number of user-article interactions in the dataset.

In [14]:
len(df.article_id.value_counts())

714

In [15]:
len(df_content.article_id.unique())

1051

In [16]:
len(df.email.dropna().unique())

5148

In [17]:
# The number of unique articles that have at least one interaction - 714
unique_articles = len(df.article_id.value_counts())

# The number of unique articles on the IBM platform - 1051
total_articles = len(df_content.article_id.unique())

# The number of unique users - 5148
unique_users = len(df.email.dropna().unique())

# The number of user-article interactions - 33669
user_article_interactions = df[['email','article_id']].shape[0]

`4.` Use the cells below to find the most viewed **article_id**, as well as how often it was viewed.  After talking to the company leaders, the `email_mapper` function was deemed a reasonable way to map users to ids.  There were a small number of null values, and it was found that all of these null values likely belonged to a single user (which is how they are stored using the function below).

In [18]:
article_views = (df.groupby(['article_id'])['email'].count())

In [19]:
article_views_df = pd.DataFrame(article_views).reset_index()

In [20]:
article_views_df.rename(columns={'email':'article_view_count'},inplace=True)

In [21]:
max_viewed = article_views_df[article_views_df.article_view_count == article_views_df.article_view_count.max()]

In [22]:
str(max_viewed.article_id.values[0])

'1429.0'

In [23]:
max_viewed.article_view_count.values[0]

937

In [24]:
# The most viewed article in the dataset as a string with one value following the decimal - 1429.0
most_viewed_article_id = str(max_viewed.article_id.values[0])

# The most viewed article in the dataset was viewed how many times? - 937
max_views = max_viewed.article_view_count.values[0]

In [25]:
## No need to change the code here - this will be helpful for later parts of the notebook
# Run this cell to map the user email to a user_id column and remove the email column

def email_mapper():
    coded_dict = dict()
    cter = 1
    email_encoded = []
    
    for val in df['email']:
        if val not in coded_dict:
            coded_dict[val] = cter
            cter+=1
        
        email_encoded.append(coded_dict[val])
    return email_encoded

email_encoded = email_mapper()
del df['email']
df['user_id'] = email_encoded

# show header
df.head()

,article_id,title,user_id
0,1430.0,"using pixiedust for fast, flexible, and easier...",1
1,1314.0,healthcare python streaming application demo,2
2,1429.0,use deep learning for image classification,3
3,1338.0,ml optimization using cognitive assistant,4
4,1276.0,deploy your python model as a restful api,5


In [26]:
## If you stored all your results in the variable names above, 
## you shouldn't need to change anything in this cell

sol_1_dict = {
    '`50% of individuals have _____ or fewer interactions.`': median_val,
    '`The total number of user-article interactions in the dataset is ______.`': user_article_interactions,
    '`The maximum number of user-article interactions by any 1 user is ______.`': max_views_by_user,
    '`The most viewed article in the dataset was viewed _____ times.`': max_views,
    '`The article_id of the most viewed article is ______.`': most_viewed_article_id,
    '`The number of unique articles that have at least 1 rating ______.`': unique_articles,
    '`The number of unique users in the dataset is ______`': unique_users,
    '`The number of unique articles on the IBM platform`': total_articles
}

# Test your dictionary against the solution
t.sol_1_test(sol_1_dict)

It looks like you have everything right here! Nice job!


### <a class="anchor" id="Rank">Part II: Rank-Based Recommendations</a>

Unlike in the earlier lessons, we don't actually have ratings for whether a user liked an article or not.  We only know that a user has interacted with an article.  In these cases, the popularity of an article can really only be based on how often an article was interacted with.

`1.` Fill in the function below to return the **n** top articles ordered with most interactions as the top. Test your function using the tests below.

In [27]:
def get_article_views_count(df=df):
    article_views = (df.groupby(['article_id'])['user_id'].count())
    
    article_views_df = pd.DataFrame(article_views).reset_index()
    
    article_views_df['article_id'] = article_views_df.article_id.astype(str)
    
    article_views_df.rename(columns={'user_id':'article_view_count'},inplace=True)
    
    return article_views_df

def get_top_articles(n, df=df):
    '''
    INPUT:
    n - (int) the number of top articles to return
    df - (pandas dataframe) df as defined at the top of the notebook 
    
    OUTPUT:
    top_articles - (list) A list of the top 'n' article titles 
    
    '''
    # Your code here
    
    top_articles = []
    
    top_articles_ids = get_top_article_ids(n, df=df)
    
    for article_id in top_articles_ids:
        top_articles.append(df[df.article_id.astype(str) == article_id]['title'].unique()[0])
    
    return top_articles # Return the top article titles from df (not df_content)

def get_top_article_ids(n, df=df):
    '''
    INPUT:
    n - (int) the number of top articles to return
    df - (pandas dataframe) df as defined at the top of the notebook 
    
    OUTPUT:
    top_articles - (list) A list of the top 'n' article titles 
    
    '''
    # Your code here
    
    article_views_df = get_article_views_count(df=df)
    
    article_views_df.article_id.astype(str)
    
    top_articles = article_views_df.sort_values(by='article_view_count',ascending=False).article_id.values[:n]
 
    return top_articles # Return the top article ids

In [28]:
print(get_top_articles(10))
print(get_top_article_ids(10))

['use deep learning for image classification', 'insights from new york car accident reports', 'visualize car data with brunel', 'use xgboost, scikit-learn & ibm watson machine learning apis', 'predicting churn with the spss random tree algorithm', 'healthcare python streaming application demo', 'finding optimal locations of new store using decision optimization', 'apache spark lab, part 1: basic concepts', 'analyze energy consumption in buildings', 'gosales transactions for logistic regression model']
['1429.0' '1330.0' '1431.0' '1427.0' '1364.0' '1314.0' '1293.0' '1170.0'
 '1162.0' '1304.0']


In [29]:
# Test your function by returning the top 5, 10, and 20 articles
top_5 = get_top_articles(5)
top_10 = get_top_articles(10)
top_20 = get_top_articles(20)

# Test each of your three lists from above
t.sol_2_test(get_top_articles)

Your top_5 looks like the solution list! Nice job.
Your top_10 looks like the solution list! Nice job.
Your top_20 looks like the solution list! Nice job.


### <a class="anchor" id="User-User">Part III: User-User Based Collaborative Filtering</a>


`1.` Use the function below to reformat the **df** dataframe to be shaped with users as the rows and articles as the columns.  

* Each **user** should only appear in each **row** once.


* Each **article** should only show up in one **column**.  


* **If a user has interacted with an article, then place a 1 where the user-row meets for that article-column**.  It does not matter how many times a user has interacted with the article, all entries where a user has interacted with an article should be a 1.  


* **If a user has not interacted with an item, then place a zero where the user-row meets for that article-column**. 

Use the tests to make sure the basic structure of your matrix matches what is expected by the solution.

In [30]:
# create the user-article matrix with 1's and 0's

def create_user_item_matrix(df):
    '''
    INPUT:
    df - pandas dataframe with article_id, title, user_id columns
    
    OUTPUT:
    user_item - user item matrix 
    
    Description:
    Return a matrix with user ids as rows and article ids on the columns with 1 values where a user interacted with 
    an article and a 0 otherwise
    '''
    # Fill in the function here
    
    unique_user_article_df = df[['user_id','article_id']].drop_duplicates()
    
    user_item = unique_user_article_df.groupby(['user_id','article_id'])[['article_id']].count().unstack()
    
    user_item.fillna(0,inplace=True)
    
    user_item = user_item.astype(dtype=int)
    
    return user_item # return the user_item matrix 

user_item = create_user_item_matrix(df)

In [31]:
## Tests: You should just need to run this cell.  Don't change the code.
assert user_item.shape[0] == 5149, "Oops!  The number of users in the user-article matrix doesn't look right."
assert user_item.shape[1] == 714, "Oops!  The number of articles in the user-article matrix doesn't look right."
assert user_item.sum(axis=1)[1] == 36, "Oops!  The number of articles seen by user 1 doesn't look right."
print("You have passed our quick tests!  Please proceed!")

You have passed our quick tests!  Please proceed!


`2.` Complete the function below which should take a user_id and provide an ordered list of the most similar users to that user (from most similar to least similar).  The returned result should not contain the provided user_id, as we know that each user is similar to him/herself. Because the results for each user here are binary, it (perhaps) makes sense to compute similarity as the dot product of two users. 

Use the tests to test your function.

In [32]:
def find_similar_users_correlation(user_id, user_item=user_item):
    '''
    INPUT:
    user_id - (int) a user_id
    user_item - (pandas dataframe) matrix of users by articles: 
                1's when a user has interacted with an article, 0 otherwise
    
    OUTPUT:
    similar_users - (list) an ordered list where the closest users (largest dot product users)
                    are listed first
    
    Description:
    Computes the similarity of every pair of users based on the dot product
    Returns an ordered
    
    '''
    # compute similarity of each user to the provided user
    correlation = {}
    
    for user_indx in user_item.index.unique():
        if user_id != user_indx:
            correlation[user_indx] = np.corrcoef(user_item.loc[user_id],user_item.loc[user_indx])[0,1]

    # sort by similarity
    similarity_df = pd.DataFrame.from_records([correlation])
    
    similarity_df = similarity_df.transpose()
    
    similarity_df.reset_index(inplace=True)
    
    similarity_df.rename(columns={'index':'user_id',0:'similarity'},inplace=True)
    
    most_similar_users = similarity_df.sort_values(by = 'similarity',ascending=False)['user_id'].values
    
    # create list of just the ids
   
    # remove the own user's id
    
    return most_similar_users # return a list of the users in order from most to least similar
        

In [33]:
def generate_similarity_df(user_id, user_item=user_item):
    # compute similarity of each user to the provided user
    dot_product = {}
    
    for user_indx in user_item.index.unique():
        if user_id != user_indx:
            dot_product[user_indx] = user_item.loc[user_id].dot(user_item.loc[user_indx].transpose())

    # sort by similarity
    similarity_df = pd.DataFrame.from_records([dot_product])
    
    similarity_df = similarity_df.transpose()
    
    similarity_df.reset_index(inplace=True)
    
    similarity_df.rename(columns={'index':'user_id',0:'similarity'},inplace=True)
    
    return similarity_df

In [34]:
def find_similar_users(user_id, user_item=user_item):
    '''
    INPUT:
    user_id - (int) a user_id
    user_item - (pandas dataframe) matrix of users by articles: 
                1's when a user has interacted with an article, 0 otherwise
    
    OUTPUT:
    similar_users - (list) an ordered list where the closest users (largest dot product users)
                    are listed first
    
    Description:
    Computes the similarity of every pair of users based on the dot product
    Returns an ordered
    
    '''
    
    similarity_df = generate_similarity_df(user_id,user_item)
    
    most_similar_users = similarity_df.sort_values(by = 'similarity',ascending=False)['user_id'].values
    
    # create list of just the ids
   
    # remove the own user's id
    
    return most_similar_users # return a list of the users in order from most to least similar
        

In [35]:
# Do a spot check of your function
print("The 10 most similar users to user 1 are: {}".format(find_similar_users(1)[:10]))
print("The 5 most similar users to user 3933 are: {}".format(find_similar_users(3933)[:5]))
print("The 3 most similar users to user 46 are: {}".format(find_similar_users(46)[:3]))

The 10 most similar users to user 1 are: [3933   23 3782  203 4459 3870  131   46 4201  395]
The 5 most similar users to user 3933 are: [   1   23 3782 4459  203]
The 3 most similar users to user 46 are: [4201   23 3782]


`3.` Now that you have a function that provides the most similar users to each user, you will want to use these users to find articles you can recommend.  Complete the functions below to return the articles you would recommend to each user. 

In [36]:
def get_article_names(article_ids, df=df):
    '''
    INPUT:
    article_ids - (list) a list of article ids
    df - (pandas dataframe) df as defined at the top of the notebook
    
    OUTPUT:
    article_names - (list) a list of article names associated with the list of article ids 
                    (this is identified by the title column)
    '''
    # Your code here
    
    article_names = df[df.article_id.isin(article_ids)].title.unique()
    
    return article_names # Return the article names associated with list of article ids


def get_user_articles(user_id, user_item=user_item):
    '''
    INPUT:
    user_id - (int) a user id
    user_item - (pandas dataframe) matrix of users by articles: 
                1's when a user has interacted with an article, 0 otherwise
    
    OUTPUT:
    article_ids - (list) a list of the article ids seen by the user
    article_names - (list) a list of article names associated with the list of article ids 
                    (this is identified by the doc_full_name column in df_content)
    
    Description:
    Provides a list of the article_ids and article titles that have been seen by a user
    '''
    # Your code here
    
    articles = user_item.loc[user_id][user_item.loc[user_id] == 1].index.values
    
    article_ids = []
    
    for ele in articles:
        _,val = ele
        article_ids.append(str(val))
        
    article_names = get_article_names(article_ids)
    
    return article_ids, article_names # return the ids and names


def user_user_recs(user_id, m=10):
    '''
    INPUT:
    user_id - (int) a user id
    m - (int) the number of recommendations you want for the user
    
    OUTPUT:
    recs - (list) a list of recommendations for the user
    
    Description:
    Loops through the users based on closeness to the input user_id
    For each user - finds articles the user hasn't seen before and provides them as recs
    Does this until m recommendations are found
    
    Notes:
    Users who are the same closeness are chosen arbitrarily as the 'next' user
    
    For the user where the number of recommended articles starts below m 
    and ends exceeding m, the last items are chosen arbitrarily
    
    '''
    # Your code here
    
    similar_user = find_similar_users(user_id=user_id)
    
    recs = []
    
    user_articles,_ = get_user_articles(user_id)
    
    for user in similar_user:
        article_ids, _ = get_user_articles(user)
        
        new_articles = list(set(article_ids) - set(user_articles))
        
        if len(recs) ==0:
            recs = new_articles
        else:
            recs = recs + new_articles
        
        if len(recs) >= m:
            break
    
    return recs[:m] # return your recommendations for this user_id    

In [37]:
# Check Results
get_article_names(user_user_recs(1, 10)) # Return 10 recommendations for user 1

array(['ml optimization using cognitive assistant',
       'got zip code data? prep it for analytics. – ibm watson data lab – medium',
       'discover hidden facebook usage insights',
       '1448    i ranked every intro to data science course on...\nName: title, dtype: object',
       'data tidying in data science experience',
       'markdown for jupyter notebooks cheatsheet',
       'flightpredict ii: the sequel  – ibm watson data lab',
       'machine learning for the enterprise',
       'why even a moth’s brain is smarter than an ai',
       'from spark ml model to online scoring with scala'], dtype=object)

In [38]:
# Test your functions here - No need to change this code - just run this cell
assert set(get_article_names(['1024.0', '1176.0', '1305.0', '1314.0', '1422.0', '1427.0'])) == set(['using deep learning to reconstruct high-resolution audio', 'build a python app on the streaming analytics service', 'gosales transactions for naive bayes model', 'healthcare python streaming application demo', 'use r dataframes & ibm watson natural language understanding', 'use xgboost, scikit-learn & ibm watson machine learning apis']), "Oops! Your the get_article_names function doesn't work quite how we expect."
assert set(get_article_names(['1320.0', '232.0', '844.0'])) == set(['housing (2015): united states demographic measures','self-service data preparation with ibm data refinery','use the cloudant-spark connector in python notebook']), "Oops! Your the get_article_names function doesn't work quite how we expect."
assert set(get_user_articles(20)[0]) == set(['1320.0', '232.0', '844.0'])
assert set(get_user_articles(20)[1]) == set(['housing (2015): united states demographic measures', 'self-service data preparation with ibm data refinery','use the cloudant-spark connector in python notebook'])
assert set(get_user_articles(2)[0]) == set(['1024.0', '1176.0', '1305.0', '1314.0', '1422.0', '1427.0'])
assert set(get_user_articles(2)[1]) == set(['using deep learning to reconstruct high-resolution audio', 'build a python app on the streaming analytics service', 'gosales transactions for naive bayes model', 'healthcare python streaming application demo', 'use r dataframes & ibm watson natural language understanding', 'use xgboost, scikit-learn & ibm watson machine learning apis'])
print("If this is all you see, you passed all of our tests!  Nice job!")

If this is all you see, you passed all of our tests!  Nice job!


`4.` Now we are going to improve the consistency of the **user_user_recs** function from above.  

* Instead of arbitrarily choosing when we obtain users who are all the same closeness to a given user - choose the users that have the most total article interactions before choosing those with fewer article interactions.


* Instead of arbitrarily choosing articles from the user where the number of recommended articles starts below m and ends exceeding m, choose articles with the articles with the most total interactions before choosing those with fewer total interactions. This ranking should be  what would be obtained from the **top_articles** function you wrote earlier.

In [39]:
def get_top_sorted_users(user_id, df=df, user_item=user_item):
    '''
    INPUT:
    user_id - (int)
    df - (pandas dataframe) df as defined at the top of the notebook 
    user_item - (pandas dataframe) matrix of users by articles: 
            1's when a user has interacted with an article, 0 otherwise
    
            
    OUTPUT:
    neighbors_df - (pandas dataframe) a dataframe with:
                    neighbor_id - is a neighbor user_id
                    similarity - measure of the similarity of each user to the provided user_id
                    num_interactions - the number of articles viewed by the user - if a u
                    
    Other Details - sort the neighbors_df by the similarity and then by number of interactions where 
                    highest of each is higher in the dataframe
     
    '''
    # Your code here
    
    similarity_df = generate_similarity_df(user_id,user_item)
    
    article_interaction_df = pd.DataFrame(df.groupby('user_id')['article_id'].count()).reset_index()
    
    article_interaction_df.rename(columns={'user_id':'neighbor_id','article_id':'num_interactions'},inplace=True)
    
    neighbors_df = similarity_df.merge(right=article_interaction_df,left_on='user_id',right_on='neighbor_id')
    
    neighbors_df.drop(columns=['user_id'],inplace=True)
    
    #neighbors_df.sort_values(by=['similarity','num_interactions'],ascending=False)['neighbor_id'].values
    
    neighbors_df = neighbors_df.sort_values(by=['similarity','num_interactions'],ascending=False)
    
    return neighbors_df # Return the dataframe specified in the doc_string


def user_user_recs_part2(user_id, m=10):
    '''
    INPUT:
    user_id - (int) a user id
    m - (int) the number of recommendations you want for the user
    
    OUTPUT:
    recs - (list) a list of recommendations for the user by article id
    rec_names - (list) a list of recommendations for the user by article title
    
    Description:
    Loops through the users based on closeness to the input user_id
    For each user - finds articles the user hasn't seen before and provides them as recs
    Does this until m recommendations are found
    
    Notes:
    * Choose the users that have the most total article interactions 
    before choosing those with fewer article interactions.

    * Choose articles with the articles with the most total interactions 
    before choosing those with fewer total interactions. 
   
    '''
    # Your code here
    
    neighbors_df = get_top_sorted_users(user_id)
    
    similar_user = neighbors_df['neighbor_id'].values[:m]
    
    recs = []
    
    user_articles,_ = get_user_articles(user_id)
    
    for user in similar_user:
        article_ids, _ = get_user_articles(user)
        
        new_articles = list(set(article_ids) - set(user_articles))
        
        if len(recs) ==0:
            recs = new_articles
        else:
            recs = recs + new_articles
        
        if len(recs) >= m:
            break
    
    recs = recs[:m]
    
    rec_names = get_article_names(recs)
    return recs, rec_names

In [40]:
# Quick spot check - don't change this code - just use it to test your functions
rec_ids, rec_names = user_user_recs_part2(20, 10)
print("The top 10 recommendations for user 20 are the following article ids:")
print(rec_ids)
print()
print("The top 10 recommendations for user 20 are the following article names:")
print(rec_names)

The top 10 recommendations for user 20 are the following article ids:
['1338.0', '1357.0', '142.0', '1157.0', '1329.0', '1427.0', '1153.0', '465.0', '164.0', '1368.0']

The top 10 recommendations for user 20 are the following article names:
['ml optimization using cognitive assistant'
 'use xgboost, scikit-learn & ibm watson machine learning apis'
 'putting a human face on machine learning'
 'airbnb data for analytics: washington d.c. listings'
 'learn tensorflow and deep learning together and now!'
 'neural networks for beginners: popular types and applications'
 'introduction to neural networks, advantages and applications'
 'airbnb data for analytics: vienna calendar'
 'overlapping co-cluster recommendation algorithm (ocular)'
 'ingest data from message hub in a streams flow']


`5.` Use your functions from above to correctly fill in the solutions to the dictionary below.  Then test your dictionary against the solution.  Provide the code you need to answer each following the comments below.

In [41]:
get_top_sorted_users(user_id=1).iloc[0]

similarity            35
neighbor_id         3933
num_interactions      45
Name: 3931, dtype: int64

In [42]:
get_top_sorted_users(user_id=131).iloc[9]

similarity           25
neighbor_id         242
num_interactions    148
Name: 240, dtype: int64

In [43]:
### Tests with a dictionary of results

user1_most_sim = 3933 # Find the user that is most similar to user 1 
user131_10th_sim = 242 # Find the 10th most similar user to user 131

In [44]:
## Dictionary Test Here
sol_5_dict = {
    'The user that is most similar to user 1.': user1_most_sim, 
    'The user that is the 10th most similar to user 131': user131_10th_sim,
}

t.sol_5_test(sol_5_dict)

This all looks good!  Nice job!


`6.` If we were given a new user, which of the above functions would you be able to use to make recommendations?  Explain.  Can you think of a better way we might make recommendations?  Use the cell below to explain a better method for new users.

__When a new user is added to the system, there is very limited information avalible regaring the user. In this scenario we can either go with Rank Based recomendation system or Content based recomendation system.__

__Rank based recomendation system will use the top k recomendations depending on article popularity.__

__Content based recomendation system will use the first article user is reading to suggest new articles (based on the description similarity). When the user has zero reading history, even content based recomendation system will trickle down to Rank based recomendation system. __

`7.` Using your existing functions, provide the top 10 recommended articles you would provide for the a new user below.  You can test your function against our thoughts to make sure we are all on the same page with how we might make a recommendation.

In [45]:
new_user = '0.0'

# What would your recommendations be for this new user '0.0'?  As a new user, they have no observed articles.
# Provide a list of the top 10 article ids you would give to 
new_user_recs = get_top_article_ids(10)# Your recommendations here



In [46]:
assert set(new_user_recs) == set(['1314.0','1429.0','1293.0','1427.0','1162.0','1364.0','1304.0','1170.0','1431.0','1330.0']), "Oops!  It makes sense that in this case we would want to recommend the most popular articles, because we don't know anything about these users."

print("That's right!  Nice job!")

That's right!  Nice job!


### <a class="anchor" id="Content-Recs">Part IV: Content Based Recommendations (EXTRA - NOT REQUIRED)</a>

Another method we might use to make recommendations is to perform a ranking of the highest ranked articles associated with some term.  You might consider content to be the **doc_body**, **doc_description**, or **doc_full_name**.  There isn't one way to create a content based recommendation, especially considering that each of these columns hold content related information.  

`1.` Use the function body below to create a content based recommender.  Since there isn't one right answer for this recommendation tactic, no test functions are provided.  Feel free to change the function inputs if you decide you want to try a method that requires more input values.  The input values are currently set with one idea in mind that you may use to make content based recommendations.  One additional idea is that you might want to choose the most popular recommendations that meet your 'content criteria', but again, there is a lot of flexibility in how you might make these recommendations.

### This part is NOT REQUIRED to pass this project.  However, you may choose to take this on as an extra way to show off your skills.

In [47]:
def tokenize(text):
    '''
    This method will:
        1. Replace any special characters with ' '.
        2. Change to lower case and trim white spaces from prefix and suffix.
        3. Remove english stop words and split string into multiple tokens.
        4. Extract stem value from each of the tokens.
        5. Lemmatize each of the tokens.

    :param text:
        String value to be tokenize.

    :return:
        Tokens extracted from String.
    '''
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    tokens = word_tokenize(text.lower().strip())
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    tokens = [PorterStemmer().stem(w) for w in tokens]
    return [WordNetLemmatizer().lemmatize(w) for w in tokens]

In [48]:
def get_tfidf_df_dot(df_content = df_content):
    content_based_df = df_content[['article_id','doc_description']]
    
    content_based_df.doc_description.fillna('',inplace=True)
    
    tfidfTransformer = TfidfVectorizer(tokenizer=tokenize)
    
    tfidf_df = tfidfTransformer.fit_transform(content_based_df.doc_description)
    
    tfidf_df_dot = tfidf_df.dot(tfidf_df.transpose())
    
    tfidf_df_dot =pd.DataFrame(tfidf_df_dot.toarray())
    
    return tfidf_df_dot

In [49]:
tfidf_df_dot = get_tfidf_df_dot()

In [50]:
def make_content_recs(user_id, curren_article, tfidf_df_dot = tfidf_df_dot, m=10):
    '''
    INPUT:
    
    OUTPUT:
    
    '''
    
    user_articles = []
    
    try:
        user_articles,_ = get_user_articles(user_id)
    
        user_articles = [int(float(i)) for i in user_articles]
    except KeyError:
        user_articles = []
    
    try:
        user_articles.append(int(float(curren_article)))
    except ValueError:
        user_articles = user_articles
    
    try:
        total_recs = tfidf_df_dot.iloc[int(float(curren_article))].sort_values(ascending=False).index.values[1:]
    except ValueError:
        total_recs = get_top_article_ids((m+len(user_articles)))
    except KeyError:
        total_recs = get_top_article_ids((m+len(user_articles)))
    except IndexError:
        total_recs = get_top_article_ids((m+len(user_articles)))
    
    recs = []
    
    for i in range(len(total_recs)):
        rec = total_recs[i]
        
        if int(float(rec)) not in user_articles:
            recs.append(rec)
            
        if len(recs) == m:
            break
            
    rec_names = get_article_names(recs)
    
    print(rec_names)
    return recs, rec_names

In [51]:
user_articles_ids,user_articles =get_user_articles(20)

In [52]:
user_articles

array(['housing (2015): united states demographic measures',
       'use the cloudant-spark connector in python notebook',
       'self-service data preparation with ibm data refinery'],
      dtype=object)

In [53]:
user_articles_ids, user_articles_ids[0]

(['232.0', '844.0', '1320.0'], '232.0')

In [54]:
make_content_recs(20,user_articles_ids[0])

['10 pieces of advice to beginner data scientists'
 'shaping data with ibm data refinery' 'tidy data in python'
 'making data science a team sport'
 'brunel interactive visualizations in jupyter notebooks'
 'the data science process' "let's have some fun with nfl data"
 'finding the user in data science'
 'a glimpse inside the mind of a data scientist'
 'breaking the 80/20 rule: how data catalogs transform data scientists’ productivity']


([766, 975, 517, 557, 892, 957, 764, 740, 115, 758],
 array(['10 pieces of advice to beginner data scientists',
        'shaping data with ibm data refinery', 'tidy data in python',
        'making data science a team sport',
        'brunel interactive visualizations in jupyter notebooks',
        'the data science process', "let's have some fun with nfl data",
        'finding the user in data science',
        'a glimpse inside the mind of a data scientist',
        'breaking the 80/20 rule: how data catalogs transform data scientists’ productivity'],
       dtype=object))

`2.` Now that you have put together your content-based recommendation system, use the cell below to write a summary explaining how your content based recommender works.  Do you see any possible improvements that could be made to your function?  Is there anything novel about your content based recommender?

### This part is NOT REQUIRED to pass this project.  However, you may choose to take this on as an extra way to show off your skills.

__Content based recomendation system is extension to Rank based recomendation system.__

__In this approach:__ 

__1. we will find out how similar two articles are by__
* Applying tokenization techniques on the article description.
* Generating TFID value.
* Applying similarity techniques on the TFID values.

__2. When a user is reading an article, to make top k recomendations:__
* Retrieve all the other articles in the order of similarity (descending).
* Remove the articles that user has already interacted with.

__3. In case if the article user is reading is new or there are no similar articles, then the top k unread from rank based recomendation are given to the user.__


`3.` Use your content-recommendation system to make recommendations for the below scenarios based on the comments.  Again no tests are provided here, because there isn't one right answer that could be used to find these content based recommendations.

In [55]:
# make recommendations for a brand new user
make_content_recs(0,'')

['healthcare python streaming application demo'
 'use deep learning for image classification'
 'apache spark lab, part 1: basic concepts'
 'predicting churn with the spss random tree algorithm'
 'analyze energy consumption in buildings'
 'visualize car data with brunel'
 'use xgboost, scikit-learn & ibm watson machine learning apis'
 'gosales transactions for logistic regression model'
 'insights from new york car accident reports'
 'finding optimal locations of new store using decision optimization']


(['1429.0',
  '1330.0',
  '1431.0',
  '1427.0',
  '1364.0',
  '1314.0',
  '1293.0',
  '1170.0',
  '1162.0',
  '1304.0'],
 array(['healthcare python streaming application demo',
        'use deep learning for image classification',
        'apache spark lab, part 1: basic concepts',
        'predicting churn with the spss random tree algorithm',
        'analyze energy consumption in buildings',
        'visualize car data with brunel',
        'use xgboost, scikit-learn & ibm watson machine learning apis',
        'gosales transactions for logistic regression model',
        'insights from new york car accident reports',
        'finding optimal locations of new store using decision optimization'],
       dtype=object))

In [56]:
# make a recommendations for a user who only has interacted with article id '1427.0'
make_content_recs(0,'1427.0')

['healthcare python streaming application demo'
 'use deep learning for image classification'
 'apache spark lab, part 1: basic concepts'
 'predicting churn with the spss random tree algorithm'
 'analyze energy consumption in buildings'
 'visualize car data with brunel'
 'gosales transactions for logistic regression model'
 'welcome to pixiedust' 'insights from new york car accident reports'
 'finding optimal locations of new store using decision optimization']


(['1429.0',
  '1330.0',
  '1431.0',
  '1364.0',
  '1314.0',
  '1293.0',
  '1170.0',
  '1162.0',
  '1304.0',
  '1436.0'],
 array(['healthcare python streaming application demo',
        'use deep learning for image classification',
        'apache spark lab, part 1: basic concepts',
        'predicting churn with the spss random tree algorithm',
        'analyze energy consumption in buildings',
        'visualize car data with brunel',
        'gosales transactions for logistic regression model',
        'welcome to pixiedust',
        'insights from new york car accident reports',
        'finding optimal locations of new store using decision optimization'],
       dtype=object))

### <a class="anchor" id="Matrix-Fact">Part V: Matrix Factorization</a>

In this part of the notebook, you will build use matrix factorization to make article recommendations to the users on the IBM Watson Studio platform.

`1.` You should have already created a **user_item** matrix above in **question 1** of **Part III** above.  This first question here will just require that you run the cells to get things set up for the rest of **Part V** of the notebook. 

In [57]:
# Load the matrix here
user_item_matrix = pd.read_pickle('user_item_matrix.p')

In [58]:
# quick look at the matrix
user_item_matrix.head()

article_id,0.0,100.0,1000.0,1004.0,1006.0,1008.0,101.0,1014.0,1015.0,1016.0,...,977.0,98.0,981.0,984.0,985.0,986.0,990.0,993.0,996.0,997.0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


`2.` In this situation, you can use Singular Value Decomposition from [numpy](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.linalg.svd.html) on the user-item matrix.  Use the cell to perform SVD, and explain why this is different than in the lesson.

In [59]:
# Perform SVD on the User-Item Matrix Here

u, s, vt = np.linalg.svd(user_item_matrix,full_matrices=True)# use the built in to get the three matrices

In [60]:
u.shape,s.shape,vt.shape

((5149, 5149), (714,), (714, 714))

__To apply Singular Value Decomposition technique, all the elements in the user_item_matrix should be populated. In the current notebook we have all the elements populated, so we were able to succesfully apply SVD on the user_item_matrix.__

__For the data used in the lesson there are missing values. Due to this reason SVD operation failed. In the situation where there are missing values in the user_item_matrix, we need to apply FunkSVD technique.__

`3.` Now for the tricky part, how do we choose the number of latent features to use?  Running the below cell, you can see that as the number of latent features increases, we obtain a lower error rate on making predictions for the 1 and 0 values in the user-item matrix.  Run the cell below to get an idea of how the accuracy improves as we increase the number of latent features.

In [61]:
num_latent_feats = np.arange(10,700+10,20)
sum_errs = []

for k in num_latent_feats:
    # restructure with k latent features
    s_new, u_new, vt_new = np.diag(s[:k]), u[:, :k], vt[:k, :]

    # take dot product
    user_item_est = np.around(np.dot(np.dot(u_new, s_new), vt_new))

    # compute error for each prediction to actual value
    diffs = np.subtract(user_item_matrix, user_item_est)

    # total errors and keep track of them
    err = np.sum(np.sum(np.abs(diffs)))
    sum_errs.append(err)

trace0 = go.Scatter(
    x = num_latent_feats,
    y = 1 - np.array(sum_errs)/df.shape[0],
    mode = 'lines',
    name = 'Accuracy'
)

layout = go.Layout(title='Accuracy vs. Number of Latent Features',
                   xaxis=dict(title='Number of Latent Features'),
                   yaxis=dict(title='Accuracy'))

fig = go.Figure(data=go.Data([trace0]),layout=layout)
iplot(fig)

`4.` From the above, we can't really be sure how many features to use, because simply having a better way to predict the 1's and 0's of the matrix doesn't exactly give us an indication of if we are able to make good recommendations.  Instead, we might split our dataset into a training and test set of data, as shown in the cell below.  

Use the code from question 3 to understand the impact on accuracy of the training and test sets of data with different numbers of latent features. Using the split below: 

* How many users can we make predictions for in the test set?  
* How many users are we not able to make predictions for because of the cold start problem?
* How many articles can we make predictions for in the test set?  
* How many articles are we not able to make predictions for because of the cold start problem?

In [62]:
df.shape

(45993, 3)

In [63]:
df_train = df.head(40000)
df_test = df.tail(5993)

def create_test_and_train_user_item(df_train, df_test):
    '''
    INPUT:
    df_train - training dataframe
    df_test - test dataframe
    
    OUTPUT:
    user_item_train - a user-item matrix of the training dataframe 
                      (unique users for each row and unique articles for each column)
    user_item_test - a user-item matrix of the testing dataframe 
                    (unique users for each row and unique articles for each column)
    test_idx - all of the test user ids
    test_arts - all of the test article ids
    
    '''
    # Your code here
    
    user_item_train = create_user_item_matrix(df_train)
    
    user_item_test = create_user_item_matrix(df_test)
    
    test_idx = list(set(user_item_test.index))
    
    test_arts = list(set(user_item_test.columns))
    
    return user_item_train, user_item_test, test_idx, test_arts

user_item_train, user_item_test, test_idx, test_arts = create_test_and_train_user_item(df_train, df_test)

In [64]:
#'How many users can we make predictions for in the test set?'
len(set(user_item_test.index) & set(user_item_train.index))

20

In [65]:
#'How many users in the test set are we not able to make predictions for because of the cold start problem?'
len(set(user_item_test.index) - set(user_item_train.index))

662

In [66]:
#'How many articles can we make predictions for in the test set?'
len(set(user_item_test.columns) & set(user_item_train.columns))

574

In [67]:
#'How many articles in the test set are we not able to make predictions for because of the cold start problem?'
len(set(user_item_test.columns) - set(user_item_train.columns))

0

In [68]:
# Replace the values in the dictionary below
a = 662 
b = 574 
c = 20 
d = 0 


sol_4_dict = {
    'How many users can we make predictions for in the test set?': c, # letter here, 
    'How many users in the test set are we not able to make predictions for because of the cold start problem?': a,  # letter here, 
    'How many movies can we make predictions for in the test set?': b,  # letter here,
    'How many movies in the test set are we not able to make predictions for because of the cold start problem?': d  # letter here
}

t.sol_4_test(sol_4_dict)

Awesome job!  That's right!  All of the test movies are in the training data, but there are only 20 test users that were also in the training set.  All of the other users that are in the test set we have no data on.  Therefore, we cannot make predictions for these users using SVD.


`5.` Now use the **user_item_train** dataset from above to find U, S, and V transpose using SVD. Then find the subset of rows in the **user_item_test** dataset that you can predict using this matrix decomposition with different numbers of latent features to see how many features makes sense to keep based on the accuracy on the test data. This will require combining what was done in questions `2` - `4`.

Use the cells below to explore how well SVD works towards making predictions for recommendations on the test data.  

In [69]:
# fit SVD on the user_item_train matrix

# fit svd similar to above then use the cells below
u_train, s_train, vt_train = np.linalg.svd(user_item_train,full_matrices=True)

In [70]:
(user_item_train.index.isin(test_idx)).sum()

20

In [71]:
(user_item_train.columns.isin(test_arts)).sum()

574

In [72]:
u_test = u_train[user_item_train.index.isin(test_idx), :]
vt_test = vt_train[:, user_item_train.columns.isin(test_arts)]
u_test.shape, vt_test.shape

((20, 4487), (714, 574))

In [73]:
test_users = list(set(user_item_train.index) & set(user_item_test.index))
test_articles = list(set(user_item_train.columns) & set(user_item_test.columns))
user_item_test_could_pred = user_item_test.loc[test_users, test_articles]

In [74]:
num_latent_feats = np.arange(0,710,20)

total_variability = np.sum(s_train**2)

proportion_variability = []

sum_errs_train = []
sum_errs_test = []

f1_train = []
f1_test = []

elapse_time = []

for k in num_latent_feats:
    start = time. time()
    # restructure with k latent features
    s_train_new, u_train_new, vt_train_new = np.diag(s_train[:k]), u_train[:, :k], vt_train[:k, :]
    u_test_new, vt_test_new = u_test[:, :k], vt_test[:k, :]
    
    proportion_variability.append(np.sum(s_train_new**2)/total_variability)
    
    # take dot product
    user_item_train_est = np.around(np.dot(np.dot(u_train_new, s_train_new), vt_train_new))
    user_item_test_est = np.around(np.dot(np.dot(u_test_new, s_train_new), vt_test_new))
    
    # compute error for each prediction to actual value
    diffs_train = np.subtract(user_item_train, user_item_train_est)
    diffs_test = np.subtract(user_item_test_could_pred, user_item_test_est)
    
    # total errors and keep track of them
    sum_errs_train.append(np.sum(np.sum(np.abs(diffs_train))))
    sum_errs_test.append(np.sum(np.sum(np.abs(diffs_test))))
    
    f1_train.append(f1_score(np.array(user_item_train).flatten(), user_item_train_est.flatten(), labels=[1.0], average='macro'))
    f1_test.append(f1_score(np.array(user_item_test_could_pred).flatten(), user_item_test_est.flatten(), labels=[1.0], average='macro'))
        
    end = time. time()
    elapse_time.append((end-start))

In [75]:
user_item_train.sum().sum()/user_item_train.size

0.009134386984122823

In [76]:
user_item_test.sum().sum()/user_item_test.size

0.011354695658393534

In [81]:
trace0 = go.Scatter(
    x = num_latent_feats,
    y = proportion_variability,
    mode = 'lines',
    name = 'Proportional Variability'
)

layout = go.Layout(title='Proportional Variability vs. Number of Latent Features',
                   xaxis=dict(title='Number of Latent Features'),
                   yaxis=dict(title='Proportional Variability'))

fig = go.Figure(data=go.Data([trace0]),layout=layout)
iplot(fig)

In [78]:
trace0 = go.Scatter(
    x = num_latent_feats,
    y = 1 - np.array(sum_errs_train)/(user_item_train.shape[0]*user_item_test.shape[1]),
    mode = 'lines',
    name = 'Training Accuracy'
)

trace1 = go.Scatter(
    x=num_latent_feats,
    y= 1 - np.array(sum_errs_test)/(user_item_test.shape[0]*user_item_test.shape[1]),
    mode = 'lines',
    name = 'Testing Accuracy'
)

trace2 = go.Scatter(
    x = num_latent_feats,
    y = elapse_time,
    mode = 'lines',
    name = 'Training Time',
    yaxis='y2'
)

layout = go.Layout(title='Accuracy & Training Time vs. Number of Latent Features',
                   xaxis=dict(title='Number of Latent Features'),
                   yaxis=dict(title='Accuracy'),
                   yaxis2=dict(
                       title='Training Time',
                       overlaying='y',
                       side='right'
                   )
                  )

fig = go.Figure(data=go.Data([trace0,trace1,trace2]),layout=layout)
iplot(fig)

In [79]:
trace0 = go.Scatter(
    x = num_latent_feats,
    y = f1_train,
    mode = 'lines',
    name = 'Training F1 Score'
)

trace1 = go.Scatter(
    x=num_latent_feats,
    y= f1_test,
    mode = 'lines',
    name = 'Testing F1 Score'
)

layout = go.Layout(title='F1 Score vs. Number of Latent Features',
                   xaxis=dict(title='Number of Latent Features'),
                   yaxis=dict(title='F1 Score')
                  )

fig = go.Figure(data=go.Data([trace0,trace1]),layout=layout)
iplot(fig)

`6.` Use the cell below to comment on the results you found in the previous question. Given the circumstances of your results, discuss what you might do to determine if the recommendations you make with any of the above recommendation systems are an improvement to how users currently find articles? 

Singular Value Decomposition technique divides the nxm user_item matrix into three matrices:

$$ U_{n x k} $$

$$\Sigma_{k x k} $$

$$V^T_{k x m} $$

where:

1. n is the number of users
2. k is the number of latent features to keep
3. m is the number of items


Sigma matrix can actually tell us how much of the original variability in the user-movie matrix is captured by each latent feature. When using all __k__ latent features, we could accurately reproduce the user-item matrix. Latent features are not all of equal importance. By discarding some of the latent features, we can reproduce the user-item matrix with minimal loss.

Our goal is to generate Sigma functions with a minimum number of latent features that can reproduce the user-item matrix with minimal loss.

For our data, when we initially performed SVD, we got 714 latent features. If we use all of the 714 latent features, we can reproduce the original user_item matrix. We need to find Sigma functions with a minimum number of latent features that can reproduce the user-item matrix with less than the threshold of loss.

To test how our Sigma function on unseen data, we have divided the data set into training (40000) and testing (5993) data sets.

We will apply the SVD technique on test data set with an incremental number of latent features. For each of the sigma matrix generated, we will calculate the loss (in terms of accuracy in the current case). 

When the training and testing loss plotted, even with zero latent features both training and testing data sets reached more than 98% accuracy. 

It is mostly because of the imbalance in the 0 and 1 class sizes in both training and testing data.

F1 score is as an alternative to accuracy for data sets with class imbalance. For the training data set, with an increase in the number of latent features F1 score has increased and reached close to 1 with all of the latent features. On the other hand, the F1 score was almost static for the testing data irrespective of Sigma matrix.

SVD is not a great approach to create a recommendation engine for the current data set. Instead, we can devise the following strategy:

1. Treat the 0 values as missing values and use FunkSVD to generate recommendation engine.

2. Use Hypothesis testing in an online environment with Null-Hypothesis as the user metrics with recommendation engine is greater than or equal to no recommendation engine, and the Alternat-Hypothesis as the user metric with recommendation engine is less than to no recommendation engine.

<a id='conclusions'></a>
### Extras
Using your workbook, you could now save your recommendations for each user, develop a class to make new predictions and update your results, and make a flask app to deploy your results.  These tasks are beyond what is required for this project.  However, from what you learned in the lessons, you certainly capable of taking these tasks on to improve upon your work here!


## Conclusion

> Congratulations!  You have reached the end of the Recommendations with IBM project! 

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the [rubric](https://review.udacity.com/#!/rubrics/2322/view). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.


## Directions to Submit

> Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations! 

In [80]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Recommendations_with_IBM.ipynb'])

0